In [1]:
import networkx as nx
import random

In [2]:
def assign_edge_delay(G, min_weight=5, max_weight=25, variation=5):
    """
    Assigns weights to edges based on node connectivity.
    Nodes with high connectivity will have lower weights, and nodes with low connectivity will have higher weights.
    The weight values are scaled between min_weight and max_weight.
    
    Parameters:
    G (networkx.Graph): The input graph.
    min_weight (int, optional): Minimum edge weight (default is 5).
    max_weight (int, optional): Maximum edge weight (default is 50).
    
    Returns:
    G (networkx.Graph): The graph with weighted edges.
    """
    node_degrees = dict(G.degree())
    min_degree = min(node_degrees.values())
    max_degree = max(node_degrees.values())
    
    for u, v in G.edges():
        avg_degree = (node_degrees[u] + node_degrees[v]) / 2
        normalized_weight = 1 - (avg_degree - min_degree) / (max_degree - min_degree)
        weight = min_weight + (max_weight - min_weight) * normalized_weight
        weight = float(round(weight))
        weight += random.randint(-variation, variation)
        if weight < min_weight:
            weight = float(min_weight)
        G[u][v]['delay'] = weight
    
    return G

In [3]:
def assign_servers(G, num_servers):
    """
    Relabels num_servers nodes as server_0, server_1, ..., selecting nodes from the edge of the network
    and placing them as closely together as possible.
    """
    node_degrees = dict(G.degree())
    edge_nodes = [node for node, degree in node_degrees.items() if degree == 1]

    # Choose a random starting edge node
    start_node = random.choice(edge_nodes)
    G.nodes[start_node]["type"] = "server_start_position"
    
    #print(edge_nodes)
    
    # Find the closest edge nodes to the starting node
    shortest_paths = nx.shortest_path_length(G, source=start_node)
    # get the num_servers closest edge nodes
    selected_nodes = [node_id for node_id, distance in shortest_paths.items() if node_id in edge_nodes][:num_servers]
    
    for i in range(len(selected_nodes)):
        G.nodes[selected_nodes[i]]["type"] = "server"
        
        if i == len(selected_nodes) - 1:
            G.nodes[selected_nodes[i]]["label"] = f"dns"
        else:
            G.nodes[selected_nodes[i]]["label"] = f"s{i}"
    
    return G

In [4]:
def assign_users(G, num_users):
    """
    Relabels num_users nodes as user_0, user_1, ..., selecting nodes from the edge of the network.
    """
    node_degrees = dict(G.degree())
    edge_nodes = [node for node, degree in node_degrees.items() if degree == 1 and G.nodes[node]["type"] != "server"]
    
    selected_nodes = random.sample(edge_nodes, num_users)
    
    for i in range(len(selected_nodes)):
        G.nodes[selected_nodes[i]]["type"] = "user"
        
        if i == len(selected_nodes) - 1:
            G.nodes[selected_nodes[i]]["label"] = f"pu"
        else:
            G.nodes[selected_nodes[i]]["label"] = f"u{i}"
        
    return G

In [5]:
def assign_relays(G, num_relays):
    """
    Relabels num_users nodes as user_0, user_1, ..., selecting nodes from the edge of the network.
    """
    node_degrees = dict(G.degree())
    relay_nodes = [node for node, degree in node_degrees.items() if G.nodes[node]["type"] != "user" and G.nodes[node]["type"] != "server" and degree <= 5]
    
    selected_nodes = random.sample(relay_nodes, num_relays)
    
    for i in range(len(selected_nodes)):
        G.nodes[selected_nodes[i]]["type"] = "relay"
        G.nodes[selected_nodes[i]]["label"] = f"r{i}"
    
    return G

In [6]:
graph = nx.barabasi_albert_graph(150, 1)
graph = assign_edge_delay(graph)

i = 0
for node in graph.nodes():
    graph.nodes[node]["type"] = "unassigned"
    graph.nodes[node]["label"] = f"z{i}"
    i+=1

graph = assign_servers(graph, num_servers=51) # one extra for DNS
graph = assign_users(graph, num_users=11) # one extra for "primary user"
graph = assign_relays(graph, num_relays=12)

In [7]:
def get_node_label(G, node_id):
    return G.nodes[node_id]["label"]

def save_as_config(G, filename):
    with open(filename, 'w') as f:
        f.write("[nodes]\n")
        for node in G.nodes():
            f.write(f"{get_node_label(G, node)}\n")
        
        f.write("[links]\n")
        for u, v in G.edges():
            f.write(f"{get_node_label(G, u)}:{get_node_label(G, v)} delay=0ms\n")

In [8]:
save_as_config(graph, "ba-topo.conf")
nx.write_graphml(graph, "barabasi_albert_graph.graphml")